In [1]:
#used to import data
import pandas as pd
#Evaluates the model
from sklearn.metrics import log_loss
import time #just used to see how long the model takes to run
from pytorch_tabnet.tab_model import TabNetRegressor

from sklearn.model_selection import train_test_split

In [2]:
df_11 = pd.read_csv('data/pbp/ready_to_model/10_11_pbp_cleaned.csv')
df_12 = pd.read_csv('data/pbp/ready_to_model/11_12_pbp_cleaned.csv')
df_13 = pd.read_csv('data/pbp/ready_to_model/12_13_pbp_cleaned.csv')
df_14 = pd.read_csv('data/pbp/ready_to_model/13_14_pbp_cleaned.csv')
df_15 = pd.read_csv('data/pbp/ready_to_model/14_15_pbp_cleaned.csv')
df_16 = pd.read_csv('data/pbp/ready_to_model/15_16_pbp_cleaned.csv')
df_17 = pd.read_csv('data/pbp/ready_to_model/16_17_pbp_cleaned.csv')
df_18 = pd.read_csv('data/pbp/ready_to_model/17_18_pbp_cleaned.csv')
df_19 = pd.read_csv('data/pbp/ready_to_model/18_19_pbp_cleaned.csv')
df_20 = pd.read_csv('data/pbp/ready_to_model/19_20_pbp_cleaned.csv')
df_21 = pd.read_csv('data/pbp/ready_to_model/20_21_pbp_cleaned.csv')
df = pd.concat([df_11,df_12,df_13,df_14,df_15,df_16,df_17,df_18,df_19,df_20,df_21])

In [3]:
def get_X_y(df):
    ''' This function takes in a dataframe and splits it into the X and y variables
    '''
    X = df.drop(['is_goal'], axis=1).to_numpy()
    y = df.is_goal.to_numpy()
    
    return X,y

X,y = get_X_y(df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.50, random_state=101)

#X_train.values
y_train = y_train.reshape(-1,1)
y_val = y_val.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [6]:
reg = TabNetRegressor()
model = reg.fit(X_train, y_train, 
                eval_set = [(X_train, y_train), (X_val, y_val)],
                eval_name=['train','val'],
                eval_metric = ['logloss'],
                max_epochs = 200,
                patience = 30)

Device used : cpu
epoch 0  | loss: 0.0581  | train_logloss: 0.21453 | val_logloss: 0.2143  |  0:00:51s
epoch 1  | loss: 0.05286 | train_logloss: 0.20298 | val_logloss: 0.20173 |  0:01:42s
epoch 2  | loss: 0.05187 | train_logloss: 0.20152 | val_logloss: 0.20008 |  0:02:32s
epoch 3  | loss: 0.0515  | train_logloss: 0.19689 | val_logloss: 0.19617 |  0:03:22s
epoch 4  | loss: 0.05129 | train_logloss: 0.1965  | val_logloss: 0.1958  |  0:04:13s
epoch 5  | loss: 0.05123 | train_logloss: 0.19855 | val_logloss: 0.19686 |  0:05:03s
epoch 6  | loss: 0.05115 | train_logloss: 0.19781 | val_logloss: 0.19739 |  0:05:53s
epoch 7  | loss: 0.05113 | train_logloss: 0.19598 | val_logloss: 0.19515 |  0:06:43s
epoch 8  | loss: 0.05126 | train_logloss: 0.19651 | val_logloss: 0.19592 |  0:07:32s
epoch 9  | loss: 0.05114 | train_logloss: 0.19553 | val_logloss: 0.19498 |  0:08:22s
epoch 10 | loss: 0.05104 | train_logloss: 0.19492 | val_logloss: 0.19441 |  0:09:12s
epoch 11 | loss: 0.051   | train_logloss: 0.196

C:\Users\Wolgy\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2279: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\Wolgy\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2279: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\Wolgy\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2279: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\Wolgy\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2279: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


epoch 14 | loss: 0.05096 | train_logloss: nan     | val_logloss: nan     |  0:12:32s
epoch 15 | loss: 0.05094 | train_logloss: 0.19513 | val_logloss: 0.19493 |  0:13:22s
epoch 16 | loss: 0.05093 | train_logloss: 0.19459 | val_logloss: 0.19423 |  0:14:12s
epoch 17 | loss: 0.05092 | train_logloss: 0.19451 | val_logloss: 0.19429 |  0:15:01s
epoch 18 | loss: 0.0509  | train_logloss: 0.19478 | val_logloss: 0.19476 |  0:15:47s
epoch 19 | loss: 0.05088 | train_logloss: 0.19411 | val_logloss: 0.19388 |  0:16:33s
epoch 20 | loss: 0.05088 | train_logloss: 0.19429 | val_logloss: 0.1943  |  0:17:19s
epoch 21 | loss: 0.0509  | train_logloss: 0.19449 | val_logloss: 0.19454 |  0:18:05s
epoch 22 | loss: 0.05086 | train_logloss: 0.19453 | val_logloss: 0.19451 |  0:18:51s
epoch 23 | loss: 0.05085 | train_logloss: 0.19523 | val_logloss: 0.19537 |  0:19:37s
epoch 24 | loss: 0.05082 | train_logloss: 0.19454 | val_logloss: 0.19474 |  0:20:23s
epoch 25 | loss: 0.05086 | train_logloss: 0.19498 | val_logloss: 

In [5]:
preds = reg.predict(X_test)
print(log_loss(y_test, preds))

0.19659255629964717


In [ ]:
#after 10 epochs logloss = .197
#after 10 epochs logloss = .197
#Added a validation set, 10 epochs logloss = .198
#With the validation set and 10 epochs, logloss = .1965 <--- slight improvement
#increased the number of steps to 5, logloss = .200
#increased the number of steps to 8, logloss = .207
#increased the number of steps to 2, logloss = .1976
#using 3 as the default fir number of steps is fine 
#Setting the n_independent to 4 results in heavy overfitting
#set n_shared to 4 and logloss increased
#Setting labmda_sparse to 2.5e-4 was slightly better, logloss = .1965
#using entmax instead of sparsemax logloss = .1976
#Increasing the batch size did have a high logloss = .1985
#however, it was still falling, I will repeat with 20 epochs and compare
#increasing the epochs did little for the performance
#Increasing the num_workers to 2 gave a logloss of .1972 but was still falling, will allow more epochs
#After 25 epochs I got logloss = .1966
#in the end the defaults worked the best. 